In [1]:
# imports
import random,string,math,csv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
all = list(csv.reader(open("../../data/training.csv","rb"), delimiter=','))

Error: iterator should return strings, not bytes (did you open the file in text mode?)

In [3]:
all = list(csv.reader(open("../../data/training.csv","r"), delimiter=','))

In [10]:
xs = np.array([list(map(float, row[1:-2])) for row in all[1:]])

In [11]:
xs.shape

(250000, 30)

In [12]:
(numPoints,numFeatures) = xs.shape

In [13]:
xs = np.add(xs, np.random.normal(0.0, 0.0001, xs.shape))

In [14]:
sSelector = np.array([row[-1] == 's' for row in all[1:]])
bSelector = np.array([row[-1] == 'b' for row in all[1:]])

In [15]:
weights = np.array([float(row[-2]) for row in all[1:]])
sumWeights = np.sum(weights)
sumSWeights = np.sum(weights[sSelector])
sumBWeights = np.sum(weights[bSelector])

In [16]:
randomPermutation = random.sample(range(len(xs)), len(xs))
numPointsTrain = int(numPoints*0.9)
numPointsValidation = numPoints - numPointsTrain

xsTrain = xs[randomPermutation[:numPointsTrain]]
xsValidation = xs[randomPermutation[numPointsTrain:]]

sSelectorTrain = sSelector[randomPermutation[:numPointsTrain]]
bSelectorTrain = bSelector[randomPermutation[:numPointsTrain]]
sSelectorValidation = sSelector[randomPermutation[numPointsTrain:]]
bSelectorValidation = bSelector[randomPermutation[numPointsTrain:]]

weightsTrain = weights[randomPermutation[:numPointsTrain]]
weightsValidation = weights[randomPermutation[numPointsTrain:]]

sumWeightsTrain = np.sum(weightsTrain)
sumSWeightsTrain = np.sum(weightsTrain[sSelectorTrain])
sumBWeightsTrain = np.sum(weightsTrain[bSelectorTrain])

In [17]:
xsTrainTranspose = xsTrain.transpose()

In [18]:
weightsBalancedTrain = np.array([0.5 * weightsTrain[i]/sumSWeightsTrain
                                 if sSelectorTrain[i]
                                 else 0.5 * weightsTrain[i]/sumBWeightsTrain\
                                 for i in range(numPointsTrain)])

In [19]:
numBins = 10

In [20]:
logPs = np.empty([numFeatures, numBins])
binMaxs = np.empty([numFeatures, numBins])
binIndexes = np.array(range(0, numPointsTrain+1, numPointsTrain/numBins))

TypeError: 'float' object cannot be interpreted as an integer

In [21]:
binIndexes = np.array(range(0, numPointsTrain+1, int(numPointsTrain/numBins)))

In [22]:
logPs = np.empty([numFeatures, numBins])
binMaxs = np.empty([numFeatures, numBins])
binIndexes = np.array(range(0, numPointsTrain+1, int(numPointsTrain/numBins)))


In [23]:
for fI in range(numFeatures):
    # index permutation of sorted feature column
    indexes = xsTrainTranspose[fI].argsort()

    for bI in range(numBins):
        # upper bin limits
        binMaxs[fI, bI] = xsTrainTranspose[fI, indexes[binIndexes[bI+1]-1]]
        # training indices of points in a bin
        indexesInBin = indexes[binIndexes[bI]:binIndexes[bI+1]]
        # sum of signal weights in bin
        wS = np.sum(weightsBalancedTrain[indexesInBin]
                    [sSelectorTrain[indexesInBin]])
        # sum of background weights in bin
        wB = np.sum(weightsBalancedTrain[indexesInBin]
                    [bSelectorTrain[indexesInBin]])
        # log probability of being a signal in the bin
        logPs[fI, bI] = math.log(wS/(wS+wB))

In [24]:
def score(x):
    logP = 0
    for fI in range(numFeatures):
        bI = 0
        # linear search for the bin index of the fIth feature
        # of the signal
        while bI < len(binMaxs[fI]) - 1 and x[fI] > binMaxs[fI, bI]:
            bI += 1
        logP += logPs[fI, bI] - math.log(0.5)
    return logP

In [25]:
def AMS(s,b):
    assert s >= 0
    assert b >= 0
    bReg = 10.
    return math.sqrt(2 * ((s + b + bReg) * 
                          math.log(1 + s / (b + bReg)) - s))

In [26]:
validationScores = np.array([score(x) for x in xsValidation])

In [27]:
tIIs = validationScores.argsort()

In [28]:
wFactor = 1.* numPoints / numPointsValidation

In [29]:
s = np.sum(weightsValidation[sSelectorValidation])
b = np.sum(weightsValidation[bSelectorValidation])

In [30]:
amss = np.empty([len(tIIs)])

In [31]:
amsMax = 0
threshold = 0.0

In [32]:
for tI in range(len(tIIs)):
    # don't forget to renormalize the weights to the same sum 
    # as in the complete training set
    amss[tI] = AMS(max(0,s * wFactor),max(0,b * wFactor))
    if amss[tI] > amsMax:
        amsMax = amss[tI]
        threshold = validationScores[tIIs[tI]]
        #print tI,threshold
    if sSelectorValidation[tIIs[tI]]:
        s -= weightsValidation[tIIs[tI]]
    else:
        b -= weightsValidation[tIIs[tI]]

In [33]:
amsMax

2.0882534929877203

In [34]:
threshold

-0.055647463568596733

In [35]:
plt.plot(amss)

In [36]:
test = list(csv.reader(open("test.csv", "r"),delimiter=','))
xsTest = np.array([list(map(float, row[1:])) for row in test[1:]])


FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [37]:
test = list(csv.reader(open("../../data/test.csv", "r"),delimiter=','))
xsTest = np.array([list(map(float, row[1:])) for row in test[1:]])


In [38]:
testIds = np.array([int(row[0]) for row in test[1:]])

In [39]:
testScores = np.array([score(x) for x in xsTest])

In [40]:
testInversePermutation = testScores.argsort()

In [41]:
testPermutation = list(testInversePermutation)
for tI,tII in zip(range(len(testInversePermutation)),
                  testInversePermutation):
    testPermutation[tII] = tI

In [42]:
submission = np.array([[str(testIds[tI]),str(testPermutation[tI]+1),
                       's' if testScores[tI] >= threshold else 'b'] 
            for tI in range(len(testIds))])

In [43]:
submission = np.append([['EventId','RankOrder','Class']],
                        submission, axis=0)

In [44]:
np.savetxt("submission.csv",submission,fmt='%s',delimiter=',')